In [1]:
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
file_path = "../data/Image_Samsung_Electronics_Sustainability_Report_2025_KOR_partial.pdf"

print("문서로드중")
loader = PyPDFLoader(file_path)
docs = loader.load()

문서로드중


In [3]:
len(docs)

30

In [4]:
docs[1]

Document(metadata={'producer': 'ezPDF Maker', 'creator': 'ezPDF Maker', 'creationdate': '2025-10-12T19:22:25+09:00', 'moddate': '2025-10-12T19:22:25+09:00', 'source': '../0_data/Image_Samsung_Electronics_Sustainability_Report_2025_KOR_partial.pdf', 'total_pages': 30, 'page': 1, 'page_label': '2'}, page_content='')

In [5]:
# uv add easyocr

In [6]:
import fitz
import numpy as np
from PIL import Image
import easyocr
from langchain.docstore.document import Document

In [7]:
def load_pdf_with_easyocr(
    pdf_path: str,
    languages=("ko","en"),   # 한국어+영어
    gpu=True,                # GPU 사용
    dpi=300,                 # OCR 품질용 렌더 DPI
    page_from=None,          # 1부터 시작, None=첫 페이지
    page_to=None             # 포함, None=마지막 페이지
):
    reader = easyocr.Reader(list(languages), gpu=gpu)
    docs = []

    with fitz.open(pdf_path) as pdf:
        total = len(pdf)
        start = 1 if page_from is None else max(1, page_from)
        end   = total if page_to is None else min(total, page_to)

        for p in range(start-1, end):
            page = pdf[p]
            zoom = dpi / 72.0
            mat = fitz.Matrix(zoom, zoom)
            pix = page.get_pixmap(matrix=mat, alpha=False)
            img = Image.frombytes("RGB", (pix.width, pix.height), pix.samples)

            arr = np.array(img)
            result = reader.readtext(arr, detail=0, paragraph=True)
            text = "\n".join(result) if result else ""

            docs.append(Document(
                page_content=text,
                metadata={"source": pdf_path, "page": p+1, "dpi": dpi, "ocr_engine": "EasyOCR"}
            ))

    return docs

In [ ]:
docs = load_pdf_with_easyocr(file_path, languages=("ko", "en"), gpu=True, dpi=300)
docs

In [10]:
len(docs)

30

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap =100
)
chunk = splitter.split_documents(docs)
len(chunk)

86